### LLMChain

In [12]:
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

llm=OpenAI(model="gpt-3.5-turbo-instruct",temperature=1)

prompt=PromptTemplate(template="{input}",input_variables=["input"])

chain=LLMChain(llm=llm,prompt=prompt,verbose=True)

In [13]:
chain("What is the meaning of life?",)



> Entering new LLMChain chain...
Prompt after formatting:
What is the meaning of life?

> Finished chain.


{'input': 'What is the meaning of life?',
 'text': "\n\nThe meaning of life is a complex and subjective concept that has been debated and pondered by philosophers, religious leaders, and individuals throughout history. It often boils down to the question of why we exist and what our purpose is on this earth. Some believe that the meaning of life is to find happiness and fulfillment, others think it is to serve a higher power or fulfill a certain destiny, and some see it as having no inherent purpose at all. Ultimately, the meaning of life may differ from person to person and can be shaped by one's beliefs, values, experiences, and goals. "}

In [14]:
chain.predict(input="给我的花店取十个名字")



> Entering new LLMChain chain...
Prompt after formatting:
给我的花店取十个名字

> Finished chain.


'：\n\n1. 花园时光\n2. 花轩\n3. 花韵居\n4. 花舍\n5. 春之馨\n6. 爱蕾花舍\n7. 静谧花房\n8. 花香浓情\n9. 花语馆\n10. 爱花阁'

In [15]:
chain.predict_and_parse(input="给我的花店取十个名字")

c:\Users\86195\.conda\envs\transformers\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(




> Entering new LLMChain chain...
Prompt after formatting:
给我的花店取十个名字

> Finished chain.


'\n\n1、花之轩\n2、乐花阁\n3、花语馨居\n4、碧落花园\n5、花漾心坊\n6、花间梦境\n7、花香小筑\n8、花语轩宇\n9、馥花居所\n10、花开时节'

### ConversationChain

In [8]:
from langchain.chains import  ConversationChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI 
from langchain.memory import ConversationBufferMemory,ConversationBufferWindowMemory,ConversationSummaryBufferMemory

llm=OpenAI(model="gpt-3.5-turbo-instruct",temperature=0)

memory=ConversationBufferMemory()
#也可以换为ConversationBufferWindowMemory(k=3),ConversationSummaryBufferMemory()

chain=ConversationChain(llm=llm,verbose=True,memory=memory) 

In [9]:
chain.run(input="你好")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好
AI:

> Finished chain.


' 你好！我是一個人工智能助手。我可以回答你的問題，或者和你聊天。你有什麼需要幫助的嗎？'

In [10]:
chain.run(input="我刚刚说了什么？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好
AI:  你好！我是一個人工智能助手。我可以回答你的問題，或者和你聊天。你有什麼需要幫助的嗎？
Human: 我刚刚说了什么？
AI:

> Finished chain.


' 你剛剛說了「你好」。這是一個常見的問候語，通常用於打招呼或開始對話。你想知道什麼其他的信息嗎？'

In [15]:
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='你好'), AIMessage(content=' 你好！我是一個人工智能助手。我可以回答你的問題，或者和你聊天。你有什麼需要幫助的嗎？'), HumanMessage(content='我刚刚说了什么？'), AIMessage(content=' 你剛剛說了「你好」。這是一個常見的問候語，通常用於打招呼或開始對話。你想知道什麼其他的信息嗎？')]))

### MultiPromptChain

笔记本演示了如何使用 RouterChain 范例来创建一个根据给定输入动态选择下一个链条的链条。

路由器链条由两个组件组成:

- RouterChain (负责选择下一个要调用的链条)
- destination_chains: 路由器链条可以路由到的链条

在本笔记本中，我们将重点讨论不同类型的路由链条。我们将展示这些路由链条在 MultiPromptChain 中的使用方式，以创建一个问答链条，该链条根据给定的问题选择最相关的提示，并使用该提示回答问题。



In [1]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [2]:
physics_template = """You are a very smart physics professor. 
You are great at answering questions about physics in a concise and easy to understand manner. 
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""

math_template = """You are a very good mathematician. You are great at answering math questions. 
You are so good because you are able to break down hard problems into their component parts, 
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

In [3]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
]

In [4]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.5)

### 创建多应用条链，以及一条默认链

In [5]:
destination_chains = {}

for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain



default_chain = ConversationChain(llm=llm, output_key="text")

In [6]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

### 创建一条路由链来导航

In [7]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [8]:
destinations_str

'physics: Good for answering questions about physics\nmath: Good for answering math questions'

In [9]:
router_template

'Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object formatted to look like:\n```json\n{{\n    "destination": string \\ name of the prompt to use or "DEFAULT"\n    "next_inputs": string \\ a potentially modified version of the original input\n}}\n```\n\nREMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.\nREMEMBER: "next_inputs" can just be the original input if you don\'t think any modifications are needed.\n\n<< CANDIDATE PROMPTS >>\nphysics: Good for answering questions about physics\nmath: Good for answering math

In [10]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [11]:
print(chain.run("What is black body radiation?"))

c:\Users\86195\.conda\envs\transformers\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Users\86195\.conda\envs\transformers\lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(




> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized object that absorbs all radiation that falls on it. This radiation is characterized by a continuous spectrum, meaning that it contains all wavelengths of light. The intensity of the radiation at each wavelength is determined by the temperature of the black body, with hotter objects emitting more radiation at shorter wavelengths. This phenomenon is important in understanding the thermal radiation emitted by objects, such as stars and planets, and plays a crucial role in many areas of physics, including thermodynamics and astrophysics. 


### Sequential

In [55]:
from langchain_openai import OpenAI,ChatOpenAI
from langchain.prompts import PromptTemplate 
from langchain.chains import SequentialChain,LLMChain


In [62]:
llm=ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=1)
template1="""你是一个玄幻小说家,对于一个小说标题,你会首先给出十个章节标题,后续再进行深度编写
对于玄幻小说标题为<{input}>的小说,请给出十个章节标题.注意章节标题不超过15个字.
格式为:1.第一章:xxx 2.第二章:xxx...."""

template2="""你是一个玄幻小说家,对于不同的小说章节标题,你会根据每一个标题,注意每一个标题都编写出200字左右的具体内容!!!
注意每一个标题都编写出200字左右的具体内容!!!
而且每一个相邻章节之间,内容必须要有关系,不能断裂.
小说的大标题是<{input}>,请根据标题:{topic},给出每一章的内容.
格式为:第一章内容:xxx  第一章内容:xxx.... """

In [63]:
template1=PromptTemplate(template=template1,input_variables=["input"])
chain_write_theme=LLMChain(llm=llm,prompt=template1,output_key="topic",verbose=True)


template2=PromptTemplate(template=template2,input_variables=["input","topic"])
chain_write_content=LLMChain(llm=llm,prompt=template2,output_key="content",verbose=True)

In [64]:
chain_write_theme.invoke("我在异世界修仙")



> Entering new LLMChain chain...
Prompt after formatting:
你是一个玄幻小说家,对于一个小说标题,你会首先给出十个章节标题,后续再进行深度编写
对于玄幻小说标题为<我在异世界修仙>的小说,请给出十个章节标题.注意章节标题不超过15个字.
格式为:1.第一章:xxx 2.第二章:xxx....

> Finished chain.


{'input': '我在异世界修仙',
 'topic': '1. 第一章: 异世界初遇\n2. 第二章: 修仙之路\n3. 第三章: 灵根觉醒\n4. 第四章: 飞升凡尘\n5. 第五章: 仙侠传承\n6. 第六章: 诛仙之战\n7. 第七章: 神秘异兽\n8. 第八章: 灵宝探秘\n9. 第九章: 仙府之门\n10. 第十章: 修真大会'}

In [65]:
chain=SequentialChain(chains=[chain_write_theme,chain_write_content],input_variables=["input"],output_variables=["content"],verbose=True)

In [66]:
result=chain({"input":"我在异世界修仙"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
你是一个玄幻小说家,对于一个小说标题,你会首先给出十个章节标题,后续再进行深度编写
对于玄幻小说标题为<我在异世界修仙>的小说,请给出十个章节标题.注意章节标题不超过15个字.
格式为:1.第一章:xxx 2.第二章:xxx....

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你是一个玄幻小说家,对于不同的小说章节标题,你会根据每一个标题,注意每一个标题都编写出200字左右的具体内容!!!
注意每一个标题都编写出200字左右的具体内容!!!
而且每一个相邻章节之间,内容必须要有关系,不能断裂.
小说的大标题是<我在异世界修仙>,请根据标题:1. 第一章: 穿越异世界
2. 第二章: 修炼之路
3. 第三章: 魔兽入侵
4. 第四章: 师门试炼
5. 第五章: 禁地探险
6. 第六章: 武道大会
7. 第七章: 隐世宗门
8. 第八章: 神兽觉醒
9. 第九章: 血脉觉醒
10. 第十章: 天地逆转,给出每一章的内容.
格式为:第一章内容:xxx  第一章内容:xxx.... 

> Finished chain.

> Finished chain.


In [67]:
result['content']

'第一章: 穿越异世界\n主人公李逍遥因意外穿越到了一个未知的异世界，对于这个陌生的世界充满好奇和兴奋，但同时也有着无比的恐惧和不安。\n\n第二章: 修炼之路\n在异世界中，修炼成为了主人公的首要任务。李逍遥开始了漫长而艰难的修炼之路，不断磨砺自己的意志和实力，探索着这个世界的奥秘与规则。\n\n第三章: 魔兽入侵\n突然，一波魔兽的入侵让整个异世界陷入了混乱和危机之中。李逍遥必须与其他修士们一起联手，抵御魔兽的侵袭，保卫自己的家园。\n\n第四章: 师门试炼\n李逍遥加入了一个修仙宗门，为了证明自己的实力和价值，他必须接受宗门的严格试炼。在试炼中，李逍遥不断突破自我，成长为一名真正的修士。\n\n第五章: 禁地探险\n宗门派遣李逍遥前往一处禁地探险，寻找传说中的宝藏和秘术。在禁地中，李逍遥面临重重危险和挑战，但也收获了许多珍贵的修炼资源和体会。\n\n第六章: 武道大会\n异世界举办了一场规模宏大的武道大会，各路修士云集一堂，展示自己的实力和修炼成就。李逍遥也参与其中，与众多高手一较高下，争夺荣耀和名声。\n\n第七章: 隐世宗门\n在一次偶然的机会中，李逍遥发现了一个神秘的隐世宗门，宗门内传承着古老的修仙秘术和精髓。他决定加入这个宗门，继承宗门的传统和使命。\n\n第八章: 神兽觉醒\n李逍遥在修炼中觉醒了自身的神兽血脉，得到了神兽的加持和力量。他的实力大幅提升，拥有了更强的战斗力和灵活性，成为众人瞩目的焦点。\n\n第九章: 血脉觉醒\n神兽血脉的觉醒让李逍遥的身体和灵魂发生了巨大的变化，他逐渐接触到了修仙之路的更深层次和秘密。他开始探索血脉的力量和本源，不断突破自身的极限。\n\n第十章: 天地逆转\n在一次大型异变中，天地之间发生了逆转和颠覆，整个世界陷入了巨大的危机和动荡之中。作为修士的李逍遥站在了救世的前线，决心逆风翻盘，守护这个美丽而危险的异世界。'